# Simulating the stochastic unbinding mechanism
In this notebook we present a simulation scheme to model the stochastic unbinding of PR molecules as proposed in our paper. 

## Main function

In [3]:
from tqdm.auto import tqdm
import numpy as np

def stochastic_unbinding(N, T, r, L,
                         D, Pu = 1,
                         kick = False, kick_size = 0,
                         surface = False):
    ''' This functions performs the simulation of a Brownian Motion Coalescence mechanism
    with stochastic unbinding of its particles. Particles inside the condensate will always
    bind with probability 1, while they have probability Pu of unbinding
    
    Args:
        :N (int): number of particles.
        :T (int): number of time steps of the simulation.
        :r (float): size of single particles.
        :L (float): size of the box acting as the environment (periodic boundary 
                    conditions apply).
        :D (float): diffusion coefficient of free particles.
        :kick (bool): if True, when particles unbind, they get kicked to avoid
                      re-binding).
        :kick_size (float): size of the kick.
        :surface (bool): If surface = True, only the particles in the surface of the 
                         droplet can escape. If False, any particle can escape (see SI
                         of our paper for details).
    Returns:
        :dist_droplet_size (list): list containing the distribution of droplet size at 
                                   each time step.
        :mean_droplet_size (array): array containing the mean size of the droplets at 
                                    each time step.
    '''
    

    ### Initial positions and droplet sizes (all free particles)
    x = np.random.rand(N, 2)*L    
    droplet_size = np.ones(N)

    ### Variables for saving data
    dist_droplet_size = []
    mean_droplet_size = np.ones(T)*N  

    ### Evolution ####
    for t in tqdm(T):
        
        label = np.arange(x.shape[0])
        max_label = label[-1]

        ### Escaping mechanism
        new_label = label.copy()
        upd_x = x.copy()
        for l in label:
            if droplet_size[l]>1:
                # Get the number of particles in the droplet
                num_parts = round(droplet_size[l]**2)
                
                # Chose the mechanism of escaping. If surface = True, only
                # the particles in the surface of the droplet can escape. If
                # False, any particle can escape (see SI of our paper for details)
                if surface:
                    surface_parts = min(num_parts, np.pi*np.sqrt(num_parts))
                    coins = np.random.rand(int(surface_parts-1))                  
                    escaped = len(coins[coins > Pu])
                
                else:
                    coins = np.random.rand(int(num_parts-1))                  
                    escaped = len(coins[coins > Pu])
                
                # Substracting escaped to the droplet size
                droplet_size[l] = np.sqrt(num_parts - escaped)
                # Heralding escaped with -1 label to avoid binding later
                droplet_size = np.append(droplet_size, np.ones(escaped))
                new_label = np.append(new_label, np.arange(escaped)+new_label[-1]+1)

                # Adding new independent positions for escaped particles. If kick = True,
                # particles receive a kick that move them away from the condensate, to 
                # further avoid re-binding in following steps
                if kick and escaped > 0:
                    angle = np.random.rand(escaped)*2*np.pi
                    news = np.vstack(x[l,0]+(np.cos(angle)*(droplet_size[l]+kick_size), # X component
                                     x[l,1]+np.sin(angle)*(droplet_size[l]+kick_size))).transpose() # Y component
                    upd_x = np.vstack((upd_x,news))
                else:                    
                    news = upd_x[l,:].repeat(escaped, axis = 0).reshape(2,escaped).transpose()
                    upd_x = np.vstack((upd_x,news))
                    
        # Updating labels and positions after unbinding
        label = new_label
        x = upd_x

        ### Calculating distances between particles
        M = np.reshape(np.repeat(x[ :, :], x.shape[0], axis = 0), (x.shape[0], x.shape[0], 2))
        Mtrans = M.transpose(1,0,2)
        distance = np.sqrt(np.square(M[:,:, 0]-Mtrans[:,:, 0])
                         + np.square(M[:,:, 1]-Mtrans[:,:, 1]))  
        # Increasing artificially distance of escaped to avoid re-binding 
        distance[label > max_label, :] = 2*L
        distance[:, label > max_label] = 2*L  

        ### Creating droplets ###
        for n, s in enumerate(droplet_size[:max_label+1]):
            distances_adapted = distance[n,:]-droplet_size
            distances_adapted[n] == 0
            close_particles = np.argwhere(distances_adapted < s*r).flatten()
            # Update the label to the one of first particle in condensate
            label[distance[n,:] < s*r] = label[n]
            

        ### Merging droplets
        u_labels = np.unique(label)
        new_droplet_size = np.zeros(len(u_labels))
        avg_pos = np.zeros((len(u_labels), 2)) 
        for idx_c, l in enumerate(u_labels):
            # Find the particles sharing label (i.e. in same droplet)
            parts_in_cond = x[label == l, :]
            size_in_cond = droplet_size[label == l]
            # Define position of droplet as the center of mass between all
            # the constituents
            avg_pos[idx_c, :] = np.mean(parts_in_cond, axis = 0)   
            # Recalculate its size
            new_droplet_size[idx_c] =  np.sqrt(np.sum(size_in_cond**2))

        ### Droplet displacement
        disp = D*np.random.randn(avg_pos.shape[0],2)                
        # Adding displacement considering stokes drag
        new_x = avg_pos + (disp.transpose()*(1/new_droplet_size)).transpose()

        
        ### Saving data for anaylysis
        dist_droplet_size.append(new_droplet_size) 
        mean_droplet_size[t] = np.mean(new_droplet_size)
        
        ### Updating position, sizes and consider periodic boundary conditions
        x = new_x
        droplet_size = new_droplet_size
        while np.max(x)>L or np.min(x)< 0: 
            x[x > L] = x[x > L]-L
            x[x < 0] = L + x[x < 0]  

               
    
    return mean_droplet_size, dist_droplet_size
    

## Results with small numbers of particles
We now showcase some of the results that can be achieved with the previous function. Note that, for the sake of not spending hours calculating, we will consider a small number of particles and short times. This means that the scalings will difer from the ones calculated in the paper, as we will suffer from finite size effects.

In [11]:

N = 80         # Number of particles
r = 1          # Radius of interaction
T = int(1e3)   # Length of simulations
S = N/0.01     # Total area, calculate as function of density
L = np.sqrt(S) # Length of the environment box
D = 1          # Diffusion coefficient of particles


reps = 36      # Number of repetitions
Pcs = (1-np.logspace(-0.1, -3, 7)) # Unbinding

In [8]:
avg_pc_short = np.zeros((len(Pcs), T))
var_pc_short = np.zeros((len(Pcs), T))

for idx, Pc in enumerate(tqdm_n(Pcs)):
    lab_pc =  round(np.log10(1-Pc), 2)  
    
    avg_res = np.array(Parallel(n_jobs=36)(delayed(droplet_changes_Pc)(N = N, T = T, r = r, L = L, D = D, Pc = Pc, rep = i, kick = True)
                for i in range(reps)), dtype = object)
    np.save(f'data_figures_response/size_N_{N}_T_{T}_S_{S}_reps_{reps}_Pc_{lab_pc}.npy', np.concatenate(avg_res[:,0]).reshape(reps, T))
    
    # Percentage of free particles
    perc = np.zeros_like(avg_res[:,1])
    for idx1, syst in enumerate(avg_res[:,1]):
        for idx2, d in enumerate(syst):
            d = d[d > 0]
            perc[idx1, idx2] = len(d[d > 1])/len(d)
      
    np.save(f'data_figures_response/perc_N_{N}_T_{T}_S_{S}_reps_{reps}_Pc_{lab_pc}.npy', perc)
    
    avg_pc_short[idx, :] = np.mean(np.concatenate(avg_res[:,0]).reshape(reps, T), axis = 0)
    var_pc_short[idx, :] = np.var(np.concatenate(avg_res[:,0]).reshape(reps, T), axis = 0)


  0%|          | 0/7 [00:00<?, ?it/s]